# All classes
## Simple general: multinomial
MedRoBERTa.nl – {“Normal_or_no_label”: Normal or no label for all classes,
                 “Present”: any non-normal present}
 
## Bespoke general: multilabel
MedRoBERTa.nl – {“Normal_or_no_label”: Normal or no label for all classes, 
                 “aortic_regurgitation_present”: ..,
                 “aortic_stenosis_present”: ..
                  …
                }
                 
# Per class, i.e. 11 models -- DONE
MedRoBERTa.nl – {“no label”: ..
                 “mild”: ..
                 “moderate”: ..
                 “severe”: ..
                “present”: ..}
 


%load_ext autoreload
%autoreload 2

In [65]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import sys
import gc
from torchinfo import summary

from typing import List, Tuple, Dict, Union
from collections import defaultdict

In [66]:
import torch

In [67]:
from datasets import Dataset, DatasetDict
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix, roc_auc_score

from transformers import TrainingArguments, Trainer
from transformers import pipeline
from transformers import DataCollatorWithPadding
from transformers import EvalPrediction

from torchinfo import summary
from transformers import AutoModelForSequenceClassification, AutoTokenizer

from sklearn.preprocessing import LabelEncoder, MultiLabelBinarizer

import benedict
import random

In [68]:
torch.manual_seed(42)
torch.cuda.manual_seed(42)
np.random.seed(42)
random.seed(42)

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [69]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("There are %d GPU(s) available." % torch.cuda.device_count())
    print("We will use the GPU:", torch.cuda.get_device_name(0))
    print(f"Memory Allocated: {torch.cuda.memory_allocated()}")
    print(f"Max memory Allocated: {torch.cuda.max_memory_allocated()}") 
    print(f"Memory reserved: {torch.cuda.memory_reserved()}")
    print(f"Max memory reserved: {torch.cuda.max_memory_reserved()}")
else:
    print("No GPU available, using the CPU instead.")
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: NVIDIA RTX 4000 Ada Generation
Memory Allocated: 1579976192
Max memory Allocated: 7143088128
Memory reserved: 5200936960
Max memory reserved: 7530872832


In [70]:
possible_classes = ["aortic_regurgitation", "aortic_stenosis", "diastolic_dysfunction", "tricuspid_regurgitation", "wma",
                    "lv_dil", "lv_syst_func", "mitral_regurgitation", "pe", "rv_dil", "rv_syst_func"]

In [71]:
deabbreviate = False
filter_reports = True
Classes = ['mitral_regurgitation','tricuspid_regurgitation', 'wma'] # 'aortic_regurgitation'
FLAG_TERMS = ['uitslag zie medische status', 'zie status', 'zie verslag status', 'slecht echovenster', 'echo overwegen', 'ge echo',
              'geen echovenster', 'geen beoordeelbaar echo', 'tee 190', 'hdf 36mnd', 'geen beoordeelbare echo', 'verslag op ic', 'psyheartanalyse']
SAVE_TERMS = ['goed', 'geen', 'normaal', 'normale']
use_multilabel = False
reduce_labels = True
num_epochs = 10
uni_model = True
uni_model_simple = False


use_multilabel = True if (uni_model) & (~uni_model_simple) else False

MULTILABELS = {'Mild': ['Mild', 'Present'], 
               'Severe': ['Severe', 'Present'],
               'Moderate': ['Moderate', 'Present'],
               'Normal': ['Normal'],
               'No label': ['No label'],
               'Present': ['Present'],
               }
Target_remapper = {'No label': 'No label', 
                   'Normal': 'Normal',
                   'Mild': 'Not normal',
                   'Moderate': 'Not normal',
                   'Severe': 'Not normal',
                   'Present': 'Not normal'
                   }



#Target_remapper = {'No label': 'Normal', 
#                   'Normal': 'Normal',
#                   'Mild': 'Not normal',
#                   'Moderate': 'Not normal',
#                   'Severe': 'Not normal',
#                   'Present':  'Not normal'
#                   }

# train a MedRoBERTa.nl model that trains on
# No label: whether there is "no label" for all classes
# Normal: whether all the classes are normal
# Present: 


In [72]:
#Add the src folder to the path
sys.path.append(os.path.abspath(os.path.join('..', 'src')))
import deabber, echo_utils

In [73]:
if deabbreviate:
    ABBREVIATIONS = benedict.benedict("../assets/abbreviations.yml")

In [74]:
plt.style.use('ggplot')
def plot_history(history, val=0):
    acc = history.history['accuracy']
    if val == 1:
        val_acc = history.history['val_accuracy'] # we can add a validation set in our fit function with nn
    loss = history.history['loss']
    if val == 1:
        val_loss = history.history['val_loss']
    x = range(1, len(acc) + 1)

    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.plot(x, acc, 'b', label='Training accuracy')
    if val == 1:
        plt.plot(x, val_acc, 'r', label='Validation accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.title('Accuracy')
    plt.legend()
    plt.subplot(1, 2, 2)
    plt.plot(x, loss, 'b', label='Training loss')
    if val == 1:
        plt.plot(x, val_loss, 'r', label='Validation loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.title('Loss')
    plt.legend()

In [75]:
def compute_metrics_binomial(logits_and_labels, averaging='macro'):
  logits, labels = logits_and_labels
  predictions = np.argmax(logits, axis=-1)
  acc = np.mean(predictions == labels)
  f1 = f1_score(labels, predictions, average = averaging)
  prec = precision_score(labels, predictions, average = averaging)
  rec = recall_score(labels, predictions, average = averaging)
  return {
          'accuracy': acc, 
          'f1_score': f1,
          'precision': prec,
          'recal': rec
          }

# source: https://jesusleal.io/2021/04/21/Longformer-multilabel-classification/
def multi_label_metrics(predictions, labels, threshold=0.5):
    # first, apply sigmoid on predictions which are of shape (batch_size, num_labels)
    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(torch.Tensor(predictions))
    # next, use threshold to turn them into integer predictions
    y_pred = np.zeros(probs.shape)
    y_pred[np.where(probs >= threshold)] = 1
    # finally, compute metrics
    y_true = labels
    
    
    f1_macro = f1_score(y_true=y_true, y_pred=y_pred, average='macro', zero_division=0)
    f1_weighted = f1_score(y_true=y_true, y_pred=y_pred, average='weighted', zero_division=0)
    f1_list = f1_score(y_true=y_true, y_pred=y_pred, average=None, zero_division=0)
    prec_macro = precision_score(y_true=y_true, y_pred=y_pred, average='macro', zero_division=0)
    prec_weighted = precision_score(y_true=y_true, y_pred=y_pred, average='weighted', zero_division=0)
    prec_list = precision_score(y_true=y_true, y_pred=y_pred, average=None, zero_division=0)
    recall_macro = recall_score(y_true=y_true, y_pred=y_pred, average='macro', zero_division=0)
    recall_weighted = recall_score(y_true=y_true, y_pred=y_pred, average='weighted', zero_division=0)
    recall_list = recall_score(y_true=y_true, y_pred=y_pred, average=None, zero_division=0)  
    
    try:
        roc_auc_weighted = roc_auc_score(y_true, probs, average = 'weighted')
        roc_auc_macro = roc_auc_score(y_true, probs, average = 'macro')
        roc_auc_list = roc_auc_score(y_true, probs, average = None)
    except:
        roc_auc_weighted = None
        roc_auc_macro = None
        roc_auc_list = []
    accuracy = accuracy_score(y_true, y_pred)
    # return as dictionary
    metrics = {'f1_macro': f1_macro,
               'f1_weighted': f1_weighted,
               'prec_weighted': prec_weighted,
               'prec_macro': prec_macro,
               'recall_macro': recall_macro,
               'recall_weighted': recall_weighted,
               'roc_auc_macro': roc_auc_macro,
               'roc_auc_weighted': roc_auc_weighted,
               'accuracy': accuracy}
    
    for k, _f1 in enumerate(f1_list):
        metrics[f'f1_{k}'] = _f1
    for k, _prec in enumerate(prec_list):
        metrics[f'prec_{k}'] = _prec
    for k, _recall in enumerate(recall_list):
        metrics[f'recall_{k}'] = _recall
    for k, _auc in enumerate(roc_auc_list):
        metrics[f'auc_{k}'] = _auc
    return metrics

def compute_metrics(p: EvalPrediction):
    preds = p.predictions[0] if isinstance(p.predictions, 
            tuple) else p.predictions
    result = multi_label_metrics(
        predictions=preds, 
        labels=p.label_ids)
    return result
     


In [76]:
def one_hot_encode(labels, num_classes):
    # labels should be a list or a 1D tensor
    one_hot = torch.zeros((len(labels), num_classes))
    rows = torch.arange(len(labels))
    one_hot[rows, labels] = 1
    return one_hot

def _mlabel_tuple_creator(x: List[int],
                          multilabels:Dict[int,List[int]],
                          num_classes: int=None)\
                          ->List[Tuple[int,...]]:
                              
    res = [(_sc for _sc in multilabels[sc]) for sc in x]
    return res

def multi_hot_encoding(x: List[int], 
                       multilabels: Union[Dict[int,List[int]], None]=None,
                       num_classes: int=None)\
                           ->torch.Tensor:    
    if multilabels is None:
        return one_hot_encode(x, num_classes=num_classes)
    else:
        return torch.Tensor(MultiLabelBinarizer(classes=range(num_classes))\
                    .fit_transform(_mlabel_tuple_creator(x,multilabels)))

# Load documents

In [77]:
tokenizer = AutoTokenizer.from_pretrained("CLTL/MedRoBERTa.nl")

C:\Users\bes3\AppData\Local\pypoetry\Cache\virtualenvs\echolabeler-DyKQcQCO-py3.10\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [78]:
os.chdir('T://lab_research/RES-Folder-UPOD/Echo_label/E_ResearchData/2_ResearchData')

In [79]:
def lab_dict_parser_simple(x: dict=None):       
    # if any of the values is not in ['Normal', 'No_label'] return 'Present'
    if any([v not in ['Normal', 'No_label'] for v in x.values()]):
        return 'Present'     
    return 'Normal_or_No_Label'

def lab_dict_parser_bespoke(x: dict=None):       
    # if any of the values is not in ['Normal', 'No_label'] return 'Present'
    if all([v in ['Normal', 'No_label'] for v in x.values()]):
        return ['Normal_or_No_Label']
    
    res_list = []
    for k,v in x.items():
        if v in ['Mild', 'Moderate', 'Severe', 'Present']:
            res_list.append(f'{k}_Present')
    res_list = list(set(res_list))
    return res_list


In [80]:
def prep_meta(df: pd.DataFrame, 
              Target_maps: Dict,
              use_multilabel: bool, 
              uni_model: bool,
              multi_labels: Dict) -> [pd.DataFrame,pd.DataFrame]:
    _multilabels = None
    # Load the train/test hashes
    test_hashes = pd.read_csv('./test_echoid.csv', sep=',')
    train_hashes = pd.read_csv('./train_echoid.csv', sep=',')
    print(train_hashes.columns)
    
    print(f"Train hashes: {train_hashes.input_hash.nunique()}")
    print(f"Test hashes: {test_hashes.input_hash.nunique()}")
    print(Target_maps.keys())
    
    
    DF = df
    DF.columns = ['sentence', 'labels', '_input_hash']
    
    if uni_model and use_multilabel:
        label_set_list = set()
        
        for l in DF['labels'].values:
            label_list = set()
            for _l in l:
                label_list.add(Target_maps[_l])
            label_set_list.add(frozenset(label_list))
            
        _multilabels = {
            k: list(s)
            for k,s in enumerate(label_set_list)
        }

        _inverse_multilabels = {
            tuple(s) : k
            for k,s in _multilabels.items()
        }        
        def get_mc_id(x):
            num_list = []
            for _x in x:
                num_list.append(Target_maps[_x])
            tlist = tuple(set(num_list))
            return _inverse_multilabels[tlist]
        
        id2label = {v:k for k,v in Target_maps.items()}
        DF= DF.assign(labels=DF['labels'].apply(get_mc_id))
        assert(DF['labels'].value_counts(dropna=True).sum() == DF.shape[0]), "Something wrong with the id mapping? Not everything is mapped it seems"

    else:    
        label2id = Target_maps
        id2label = {v:k for k,v in label2id.items()}
        num_labels = len(label2id)
        DF['labels'] = DF['labels'].map(label2id)

    
    if filter_reports:
        DF = DF.assign(sentence = echo_utils.report_filter(DF.sentence, 
                                                flag_terms=FLAG_TERMS, 
                                                save_terms=SAVE_TERMS)[0])
        DF = DF.loc[DF.sentence.notna()]
    
    if deabbreviate:
        DeAbber = deabber.deabber(model_type='sbert', 
                                  abbreviations=ABBREVIATIONS['nl']['echocardiogram'], 
                                  min_sim=0.5, 
                                  top_k=10)
        DF = DF.assign(sentence=DeAbber.deabb(DF.sentence.values, TokenRadius=3))
        
    if use_multilabel:
        if _multilabels is None:
            if (multi_labels is not None):
                _multilabels = {label2id[k]: [label2id[l] for l in v]
                                for k,v in multi_labels.items()}
            else:
                raise(ValueError, "No multilabels provided")            
    else:
        _multilabels = None
        
    
    # TODO: make proper
    DFtrain = DF.loc[DF._input_hash.isin(train_hashes.input_hash.unique()), ['sentence', 'labels']]
    DFtest = DF.loc[DF._input_hash.isin(test_hashes.input_hash.unique()), ['sentence', 'labels']]
    
    print("Train labels:")
    print(DFtrain.labels.value_counts())
    print("Test labels:")
    print(DFtest.labels.value_counts())   

    return DFtrain, DFtest, id2label, _multilabels

def tokenize_meta(HFDataSet, tokenizer, num_labels, multilabels):
    Tokenized_DataSet = HFDataSet.map(lambda batch: tokenizer(batch, truncation=True, 
                                                                      padding=True, 
                                                                      max_length=256),
                                      input_columns='sentence',
                                      batched=True,
                                      remove_columns=['sentence'])
    
    Tokenized_DataSet = (Tokenized_DataSet
                          #.map(lambda x : {"float_labels": x["labels"].to(torch.float)}, remove_columns=["labels"])
                          .map(lambda x: {"labels": 
                              multi_hot_encoding(x['labels'], 
                                                 multilabels=multilabels, 
                                                 num_classes=num_labels)}, 
                               batched=True, remove_columns=['labels']))                      
                          #.rename_column("float_labels", "labels"))
    
    Tokenized_DataSet.set_format("torch", 
                                 columns=['input_ids', 'attention_mask', 'labels'])
    
    data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
    return Tokenized_DataSet, data_collator

def get_trainer(compute_metrics=None, DataCollator=None, tokenizer=None, TokenizedDataSet=None, Target_maps=None, id2label=None, num_epochs=5):
    # https://colab.research.google.com/drive/1aue7x525rKy6yYLqqt-5Ll96qjQvpqS7#scrollTo=1eVCRpcLUW-y
    # https://github.com/NielsRogge/Transformers-Tutorials/blob/master/BERT/Fine_tuning_BERT_(and_friends)_for_multi_label_text_classification.ipynb
    
    print(f"Target map: {Target_maps}")
    print(f"id2label: {id2label}")
    
    medroberta_clf = AutoModelForSequenceClassification.from_pretrained("CLTL/MedRoBERTa.nl", 
                                                    num_labels=len(id2label.keys()),
                                                    problem_type='multi_label_classification',
                                                    id2label=id2label,
                                                    label2id=Target_maps)
    
    train_dir = "T:\\laupodteam/AIOS/Bram/data/tmp"
    metric_name = 'f1_macro'
    training_args = TrainingArguments(output_dir=train_dir,
                                      evaluation_strategy='epoch',
                                      save_strategy='epoch',
                                      num_train_epochs=num_epochs,
                                      learning_rate=5e-5,
                                      per_device_train_batch_size=16,
                                      weight_decay=0.01,
                                      per_device_eval_batch_size=10,
                                      load_best_model_at_end=True, 
                                      metric_for_best_model=metric_name)
    
    trainer = Trainer(medroberta_clf,
                      training_args,
                      train_dataset = TokenizedDataSet["train"],
                      eval_dataset = TokenizedDataSet["test"],
                      tokenizer=tokenizer,
                      data_collator=DataCollator,
                      compute_metrics=compute_metrics)
    
    return trainer, medroberta_clf

In [81]:
# inv_Target_maps = {v:k for k,v in Target_maps.items()}
# 
# ClassCount = defaultdict(int)
# for k,v in _multilabels.items():
#     for _v in v:
#         ClassCount[_v] += 1
#         
# test_label_count = defaultdict(int)
# for l in DFtest.labels:
#     for _l in _multilabels[l]:
#         test_label_count[inv_Target_maps[_l]] += 1
# sorted(test_label_count.items(), key=lambda x: x[1], reverse=True)

In [82]:
# test_label_count_str = defaultdict(int)
# for l in labeled_documents.loc[labeled_documents._input_hash.isin(test_hashes.input_hash.unique()), 'labels']:
#     for _l in l:
#         test_label_count_str[_l] += 1
# sorted(test_label_count_str.items(), key=lambda x: x[1], reverse=True)

In [83]:
if uni_model==True:
    # load merged..
    labeled_documents = pd.read_json(f"./echo_doc_labels/merged_labels.jsonl", lines=True)
    if uni_model_simple: # multiclass
        print("Prepping uni model, simple...")
        labeled_documents['label'] = labeled_documents['labels'].apply(lab_dict_parser_simple)
        labeled_documents = labeled_documents.drop('labels', axis=1)
        labeled_documents=labeled_documents[['text', 'label', '_input_hash']]
        Target_maps = {Label:i for i,Label in enumerate(labeled_documents['label'].unique())}
        num_labels = len(Target_maps)
    else: # multilabel
        print("Prepping uni model, bespoke...")
        labeled_documents['label'] = labeled_documents['labels'].apply(lab_dict_parser_bespoke)
        labeled_documents = labeled_documents.assign(label=labeled_documents['label'].apply(lambda x: list(set(x))))
        labeled_documents = labeled_documents.drop('labels', axis=1)
        labeled_documents=labeled_documents[['text', 'label', '_input_hash']]        
        _labels = set()
        for l in labeled_documents['label']:
            for _l in l:
                _labels.add(_l)
        
        Target_maps = {Label:i for i,Label in enumerate(_labels)}
        num_labels = len(Target_maps)
    
    
    DFtrain, DFtest, id2label, _multilabels = prep_meta(labeled_documents, Target_maps, 
                                                        use_multilabel=use_multilabel, 
                                                        uni_model=uni_model,
                                                        multi_labels=None)
    
    TrainSet = Dataset.from_pandas(DFtrain)
    TestSet = Dataset.from_pandas(DFtest)
    
    HF_DataSet = DatasetDict(
        {'train' : TrainSet,
         'test': TestSet,
        }
    )
    
    TokenizedDataSet, DataCollator = tokenize_meta(HF_DataSet, tokenizer, num_labels, _multilabels)
    trainer, medroberta_clf = get_trainer(compute_metrics, DataCollator, tokenizer, TokenizedDataSet, Target_maps, id2label, num_epochs)   

    trainer.train()
    trainer.eval_dataset = TokenizedDataSet["test"]
    _metrics = trainer.evaluate()
    
    reduced_label_str = "simple" if uni_model_simple else "bespoke"
    mod_string = f"MultiModel_Epochs{num_epochs}_{reduced_label_str}"   
    new_dir = f"T://lab_research/RES-Folder-UPOD/Echo_label/G_Output/3_OtherOutput/MedRoBERTa/{mod_string}"
    os.mkdir(new_dir)

    trainer.save_model(new_dir)
    trainer.save_metrics(new_dir, metrics=_metrics)
    
    torch.cuda.empty_cache()
    gc.collect()

elif uni_model==False:
    for Class in Classes:
        labeled_documents = pd.read_json(f"./echo_doc_labels/{Class}.jsonl", lines=True)
        print(f"Label counts: {labeled_documents.label.value_counts()}")
        if reduce_labels:
            labeled_documents['label'] = labeled_documents.label.map(Target_remapper)
            print(f"Remapped labels: {labeled_documents.label.value_counts(dropna=False)}")
        # Expand with label columns
        Target_maps = {Label:i for i,Label in enumerate(labeled_documents['label'].unique())}
        num_labels = len(Target_maps)
        
        DFtrain, DFtest, id2label, _multilabels = prep_meta(labeled_documents, Target_maps, 
                                                            use_multilabel=use_multilabel,
                                                            multi_labels=MULTILABELS)
        
        TrainSet = Dataset.from_pandas(DFtrain)
        TestSet = Dataset.from_pandas(DFtest)
        
        HF_DataSet = DatasetDict(
            {'train' : TrainSet,
             'test': TestSet,
            }
        )
        
        TokenizedDataSet, DataCollator = tokenize_meta(tokenizer, num_labels, _multilabels)
        
        trainer, medroberta_clf = get_trainer(compute_metrics, DataCollator, tokenizer, TokenizedDataSet, Target_maps, id2label, num_epochs)
    
        
        trainer.train()
        trainer.eval_dataset = TokenizedDataSet["test"]
        _metrics = trainer.evaluate()
        
        
        reduced_label_str = "reduced_labels" if reduce_labels else "full_labels"
        mod_string = f"{Class}_Epochs{num_epochs}_{reduced_label_str}"
        new_dir = f"T://lab_research/RES-Folder-UPOD/Echo_label/G_Output/3_OtherOutput/MedRoBERTa/{mod_string}"
        os.mkdir(new_dir)
        trainer.save_model(new_dir)
        trainer.save_metrics(new_dir, metrics=_metrics)
        
        # clear GPU memory 
        del trainer, medroberta_clf    
        torch.cuda.empty_cache()
        gc.collect()    trainer.save_model(new_dir)
    trainer.save_metrics(new_dir, metrics=_metrics)
    

Prepping uni model, bespoke...
Index(['ECHO_StudyID', 'ECHO_StudyID.1', 'input_hash', 'task_hash'], dtype='object')
Train hashes: 96026
Test hashes: 24051
dict_keys(['pe_Present', 'rv_dil_Present', 'wma_Present', 'lv_dil_Present', 'aortic_valve_native_stenosis_Present', 'mitral_valve_native_regurgitation_Present', 'lv_sys_func_Present', 'rv_sys_func_Present', 'aortic_valve_native_regurgitation_Present', 'lv_dias_func_Present', 'Normal_or_No_Label', 'tricuspid_valve_native_regurgitation_Present'])
Train labels:
labels
181    1315
155     207
74      203
215     146
45      115
       ... 
20        1
291       1
302       1
171       1
97        1
Name: count, Length: 313, dtype: int64
Test labels:
labels
181    346
155     47
74      47
45      34
261     31
      ... 
69       1
18       1
58       1
19       1
252      1
Name: count, Length: 150, dtype: int64


Map:   0%|          | 0/3909 [00:00<?, ? examples/s]

Map:   0%|          | 0/958 [00:00<?, ? examples/s]

Map:   0%|          | 0/3909 [00:00<?, ? examples/s]

Map:   0%|          | 0/958 [00:00<?, ? examples/s]

Target map: {'pe_Present': 0, 'rv_dil_Present': 1, 'wma_Present': 2, 'lv_dil_Present': 3, 'aortic_valve_native_stenosis_Present': 4, 'mitral_valve_native_regurgitation_Present': 5, 'lv_sys_func_Present': 6, 'rv_sys_func_Present': 7, 'aortic_valve_native_regurgitation_Present': 8, 'lv_dias_func_Present': 9, 'Normal_or_No_Label': 10, 'tricuspid_valve_native_regurgitation_Present': 11}
id2label: {0: 'pe_Present', 1: 'rv_dil_Present', 2: 'wma_Present', 3: 'lv_dil_Present', 4: 'aortic_valve_native_stenosis_Present', 5: 'mitral_valve_native_regurgitation_Present', 6: 'lv_sys_func_Present', 7: 'rv_sys_func_Present', 8: 'aortic_valve_native_regurgitation_Present', 9: 'lv_dias_func_Present', 10: 'Normal_or_No_Label', 11: 'tricuspid_valve_native_regurgitation_Present'}


C:\Users\bes3\AppData\Local\pypoetry\Cache\virtualenvs\echolabeler-DyKQcQCO-py3.10\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at CLTL/MedRoBERTa.nl and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,F1 Macro,F1 Weighted,Prec Weighted,Prec Macro,Recall Macro,Recall Weighted,Roc Auc Macro,Roc Auc Weighted,Accuracy,F1 0,F1 1,F1 2,F1 3,F1 4,F1 5,F1 6,F1 7,F1 8,F1 9,F1 10,F1 11,Prec 0,Prec 1,Prec 2,Prec 3,Prec 4,Prec 5,Prec 6,Prec 7,Prec 8,Prec 9,Prec 10,Prec 11,Recall 0,Recall 1,Recall 2,Recall 3,Recall 4,Recall 5,Recall 6,Recall 7,Recall 8,Recall 9,Recall 10,Recall 11,Auc 0,Auc 1,Auc 2,Auc 3,Auc 4,Auc 5,Auc 6,Auc 7,Auc 8,Auc 9,Auc 10,Auc 11
1,No log,0.159194,0.637728,0.776969,0.845254,0.759917,0.584546,0.750000,0.939218,0.963314,0.590814,0.000000,0.317460,0.824427,0.429630,0.637681,0.770686,0.946844,0.542373,0.670103,0.891641,0.937677,0.684211,0.000000,0.769231,0.964286,0.659091,0.846154,0.657258,0.950000,0.827586,0.802469,0.947368,0.919444,0.776119,0.000000,0.200000,0.720000,0.318681,0.511628,0.931429,0.943709,0.403361,0.575221,0.842105,0.956647,0.611765,0.750774,0.916520,0.960611,0.898779,0.974203,0.975238,0.992913,0.926543,0.951940,0.978117,0.983552,0.961431
2,No log,0.095848,0.808628,0.879381,0.938873,0.930650,0.737545,0.840692,0.982975,0.987126,0.732777,0.476190,0.615385,0.880282,0.815287,0.750000,0.919220,0.956229,0.673575,0.904110,0.904025,0.955903,0.853333,0.909091,0.857143,0.932836,0.969697,0.931034,0.896739,0.972603,0.878378,0.933962,0.960526,0.941176,0.984615,0.322581,0.480000,0.833333,0.703297,0.627907,0.942857,0.940397,0.546218,0.876106,0.853801,0.971098,0.752941,0.966489,0.962885,0.979629,0.978542,0.991613,0.994118,0.993398,0.961318,0.992711,0.987383,0.993667,0.993949
3,0.199100,0.074570,0.898370,0.929387,0.935785,0.917956,0.882313,0.924224,0.986289,0.989639,0.821503,0.763636,0.752475,0.907850,0.909091,0.900000,0.952113,0.963696,0.873469,0.950226,0.928571,0.962644,0.916667,0.875000,0.745098,0.930070,0.941176,0.972973,0.938889,0.960526,0.849206,0.972222,0.945455,0.957143,0.927711,0.677419,0.760000,0.886667,0.879121,0.837209,0.965714,0.966887,0.899160,0.929204,0.912281,0.968208,0.905882,0.954171,0.972401,0.984587,0.983219,0.997408,0.994578,0.993191,0.979317,0.995612,0.994107,0.990314,0.996564
4,0.199100,0.068588,0.901882,0.931510,0.939181,0.909999,0.895535,0.925418,0.987983,0.990752,0.824635,0.779661,0.725490,0.892086,0.922222,0.909091,0.953425,0.963087,0.893617,0.950673,0.944282,0.959538,0.929412,0.821429,0.711538,0.968750,0.932584,0.888889,0.915789,0.976190,0.905172,0.963636,0.947059,0.959538,0.929412,0.741935,0.740000,0.826667,0.912088,0.930233,0.994286,0.950331,0.882353,0.938053,0.941520,0.959538,0.929412,0.971883,0.966079,0.991180,0.984334,0.998297,0.994439,0.993251,0.978416,0.994083,0.992800,0.992841,0.998194
5,0.061000,0.064220,0.905146,0.934501,0.945944,0.927072,0.886369,0.924821,0.987168,0.990341,0.834029,0.771930,0.760870,0.900709,0.917127,0.883721,0.947658,0.973422,0.895652,0.959641,0.935294,0.958153,0.957576,0.846154,0.833333,0.962121,0.922222,0.883721,0.914894,0.976667,0.927928,0.972727,0.940828,0.956772,0.987500,0.709677,0.700000,0.846667,0.912088,0.883721,0.982857,0.970199,0.865546,0.946903,0.929825,0.959538,0.929412,0.964262,0.963833,0.991469,0.984157,0.998195,0.997139,0.994412,0.982662,0.993067,0.986900,0.991027,0.998895
6,0.061000,0.060129,0.911228,0.938428,0.940141,0.911277,0.911992,0.937351,0.989039,0.991120,0.844468,0.819672,0.772277,0.920962,0.917127,0.888889,0.958449,0.971714,0.880342,0.954955,0.945559,0.963610,0.941176,0.833333,0.764706,0.950355,0.922222,0.851064,0.930108,0.976589,0.895652,0.972477,0.926966,0.970674,0.941176,0.806452,0.780000,0.893333,0.912088,0.930233,0.988571,0.966887,0.865546,0.938053,0.964912,0.956647,0.941176,0.968612,0.972269,0.994810,0.984942,0.997992,0.994592,0.994437,0.982993,0.996335,0.994628,0.987788,0.999070
7,0.029000,0.060926,0.917705,0.940102,0.943630,0.926551,0.909804,0.937351,0.988704,0.991096,0.849687,0.800000,0.804124,0.916667,0.917127,0.928571,0.947945,0.970000,0.902128,0.968889,0.938776,0.959654,0.958580,0.827586,0.829787,0.956522,0.922222,0.951220,0.910526,0.976510,0.913793,0.973214,0.936047,0.956897,0.964286,0.7741

FileExistsError: [WinError 183] Kan geen bestand maken dat al bestaat: 'T://lab_research/RES-Folder-UPOD/Echo_label/G_Output/3_OtherOutput/MedRoBERTa/MultiModel_Epochs10_bespoke'

In [85]:
Target_maps

{'pe_Present': 0,
 'rv_dil_Present': 1,
 'wma_Present': 2,
 'lv_dil_Present': 3,
 'aortic_valve_native_stenosis_Present': 4,
 'mitral_valve_native_regurgitation_Present': 5,
 'lv_sys_func_Present': 6,
 'rv_sys_func_Present': 7,
 'aortic_valve_native_regurgitation_Present': 8,
 'lv_dias_func_Present': 9,
 'Normal_or_No_Label': 10,
 'tricuspid_valve_native_regurgitation_Present': 11}

In [ ]:
# consider quantisation using bitsandbytes or quanto